In [ ]:
pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.4/294.4 kB 30.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.25.0 which is incompatible.


In [1]:
from google.colab import files
uploaded = files.upload()

Saving 공원별수요지수.csv to 공원별수요지수.csv


In [2]:
from google.colab import files
uploaded = files.upload()

Saving 접근성지수(공원).csv to 접근성지수(공원).csv


In [3]:
from google.colab import files
uploaded = files.upload()

Saving 인프라지수(공원).csv to 인프라지수(공원).csv


In [8]:
from google.colab import files
uploaded = files.upload()

Saving numdata(park).csv to numdata(park).csv


In [ ]:
import pandas as pd
import numpy as np

def calculate_coverage(selected_facilities, demand_radius, locations):
    covered_indices = set()
    for facility_index in selected_facilities:
        facility_location = locations[facility_index]
        within_coverage = np.linalg.norm(locations - facility_location, axis=1) <= demand_radius
        covered_indices.update(np.where(within_coverage)[0])
    return covered_indices

merged_df = pd.read_csv('numdata(park).csv', encoding="CP949")

# 데이터 프레임에서 위치 정보 추출 (위도, 경도)
locations = merged_df[['위도', '경도', '공원명']].values

# MCLP 알고리즘을 위한 데이터 변환
data = merged_df[['demandNum', 'infraNum', 'accNum', '경도']].values

# Combine all three criteria into a single weight (customize this logic)
weights = 0.5 * data[:, 0] + 0.3 * data[:, 1] + 0.2 * data[:, 2]

# MCLP 알고리즘 적용
num_facilities = 6  # 선택할 놀이터의 수
demand_radius = 0.0063  # Covering radius for demand points

selected_facilities = []
remaining_indices = set(range(len(data)))

for _ in range(num_facilities):
    # 남은 인덱스에 대해 각각의 가중치 합 및 커버링된 인덱스 계산
    total_weights = np.sum(weights[list(remaining_indices)])
    covered_indices = calculate_coverage(selected_facilities, demand_radius, locations)
    uncovered_indices = remaining_indices - covered_indices

    if not uncovered_indices:  # Check if uncovered_indices is empty
        break

    # 최대 가중치를 가진 인덱스 선택
    selected_index = max(uncovered_indices, key=lambda i: weights[i])
    # 선택된 인덱스를 결과에 추가하고 남은 인덱스에서 제거
    selected_facilities.append(selected_index)
    remaining_indices.remove(selected_index)

# 결과를 데이터프레임으로 변환
result_df = merged_df.iloc[selected_facilities]

# 결과를 CSV 파일로 저장
result_df.to_csv('selected_facilities.csv', index=False)

# 결과 출력
print("Selected Facility IDs:", selected_facilities)
print("Selected Facility Locations:", locations[selected_facilities])


Selected Facility IDs: [60, 70, 7, 2, 91, 47]
Selected Facility Locations: [[ 37.524624 126.856207]
 [ 37.521958 126.867486]
 [ 37.535258 126.868666]
 [ 37.541779 126.864344]
 [ 37.50836  126.869014]
 [ 37.521716 126.875912]]


In [13]:
import pandas as pd
import numpy as np

def calculate_coverage(selected_facilities, demand_radius, locations, excluded_indices):
    covered_indices = set()
    for facility_index in selected_facilities:
        if facility_index in excluded_indices:
            continue
        facility_location = locations[facility_index]
        within_coverage = np.linalg.norm(locations - facility_location, axis=1) <= demand_radius
        # Filter out excluded indices
        covered_indices.update([i for i in np.where(within_coverage)[0] if i not in excluded_indices])
    return covered_indices

merged_df = pd.read_csv('numdata(park).csv')

# 데이터 프레임에서 위치 정보 추출 (위도, 경도)
locations = merged_df[['위도', '경도']].values

# MCLP 알고리즘을 위한 데이터 변환
data = merged_df[['demandNum', 'infraNum', 'accNum', '경도']].values

# Combine all three criteria into a single weight
weights = 0.5 * data[:, 0] + 0.3 * data[:, 1] + 0.2 * data[:, 2]

# 제외할 공원들의 이름
excluded_parks = ['용왕산공원','안양천공원','으뜸가로공원','안양천 가족정원','목동마중숲', '갈산공원','오솔길공원']
excluded_indices = merged_df[merged_df['공원명'].isin(excluded_parks)].index

# 기존 놀이터 위치
existing_facility_locations = locations[excluded_indices]

# MCLP 알고리즘 적용
num_facilities = 6  # 선택할 놀이터의 수
demand_radius = 0.0100  # Covering radius for demand points

selected_facilities = []
remaining_indices = set(range(len(data))) - set(excluded_indices)

for _ in range(num_facilities):
    total_weights = np.sum(weights[list(remaining_indices)])
    covered_indices = calculate_coverage(selected_facilities, demand_radius, locations, excluded_indices)
    uncovered_indices = remaining_indices - covered_indices

    if not uncovered_indices:
        break

    # 최대 가중치를 가진 인덱스 선택을 위한 초기화
    selected_index = None
    max_weight = -1

    # 각 후보 인덱스에 대해 검사
    for candidate_index in uncovered_indices:
        candidate_location = locations[candidate_index]
        # 기존 놀이터와의 거리를 확인
        distances_to_existing = np.linalg.norm(existing_facility_locations - candidate_location, axis=1)
        if np.all(distances_to_existing > demand_radius):
            # 기준 거리 이내에 기존 놀이터가 없는 경우에만 선택
            if weights[candidate_index] > max_weight:
                selected_index = candidate_index
                max_weight = weights[candidate_index]

    if selected_index is not None:
        selected_facilities.append(selected_index)
        remaining_indices.remove(selected_index)

# 결과를 데이터프레임으로 변환
result_df = merged_df.iloc[selected_facilities]

# 결과를 CSV 파일로 저장
result_df.to_csv('selected_facilities.csv', index=False)

# 결과 출력
print("Selected Facility IDs:", selected_facilities)
print("Selected Facility Locations:", locations[selected_facilities])



Selected Facility IDs: [67, 7, 57, 90, 3, 48]
Selected Facility Locations: [[ 37.521958 126.867486]
 [ 37.535258 126.868666]
 [ 37.524624 126.856207]
 [ 37.517649 126.848293]
 [ 37.545207 126.865033]
 [ 37.511144 126.839205]]


In [ ]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # 지구의 반지름 (킬로미터)

    # 위도 및 경도를 도에서 라디안으로 변환
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # 좌표의 차이
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # 하버사인 공식
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c

    return distance

# 예시 사용법
lat, lon = 37.5665, 126.9780  # 예시 좌표 (서울)
covering_radius = 0.0065  # 커버링 반경 (라디안으로 표현)

# 위도, 경도에서 0.01 라디안에 해당하는 거리 계산
radius_distance = haversine(lat, lon, lat + covering_radius, lon)

print(f"커버링 반경 0.01의 경우, 약 {radius_distance:.2f} 킬로미터 입니다.")


커버링 반경 0.01의 경우, 약 0.72 킬로미터 입니다.


In [6]:
import pandas as pd

# 파일 로드
accessibility_df = pd.read_csv('접근성지수(공원).csv')
demand_df = pd.read_csv('공원별수요지수.csv')
infrastructure_df = pd.read_csv('인프라지수(공원).csv')

# 필요한 컬럼 선택
accessibility_df = accessibility_df[['위도', '경도', '정규화된 접근성 지수']]
demand_df = demand_df[['위도', '경도', '공원명', '수요 지수']]
infrastructure_df = infrastructure_df[['위도', '경도', 'infraNum']]

# 위도와 경도를 기준으로 세 데이터 프레임 병합
merged_df = pd.merge(accessibility_df, demand_df, on=['위도', '경도'], how='inner')
final_df = pd.merge(merged_df, infrastructure_df, on=['위도', '경도'], how='inner')

# 컬럼 이름 변경
final_df = final_df.rename(columns={'정규화된 접근성 지수': 'accNum', '수요 지수': 'demandNum'})

# '공원명' 열을 맨 뒤로 이동
cols = final_df.columns.tolist()
cols = [col for col in cols if col != '공원명'] + ['공원명']
final_df = final_df[cols]

# 결과 저장
final_df.to_csv('merged_data.csv', index=False)


